# Taller 1 - Python en la práctica

En la clase 1 de AABC se introdujeron conceptos básicos de Python y del aprendizaje automático. Luego, se introdujo el set de datos **Iris** para armar su primer clasificador.

El objetivo de este taller es poder experimentar un poco con Python para ir ganando confianza con esta herramienta, mientras vamos explorarando algunos otros aspectos del set de datos Iris. Aprovechen para modificar el codigo y probar!

**Nota:** Muchas funciones usadas aquí son copiadas del notebook 3 de la Clase 1. No copiamos las explicaciones de las mismas, pero puede ir a buscarlas al notebook original.

**Nota2:** Dejamos celdas vacias a lo largo del notebook para que puedan responder aqui mismo. Esperamos que las respuestas a este taller sean un notebook como este en formato _.pdf_.

In [ ]:
# Comenzamos importando los paquetes necesarios, y cargando el set de datos

import numpy as np # paquete con funcionalidades matemáticas
from scipy import stats # paquete con funcionalidades estadísticas
from sklearn import datasets # paquete con sets de datos
from sklearn import model_selection # paquete que divide train y test set
import pandas # paquete que permite manipular datos con formato de tabla
from pandas import plotting as pandas_plot # funciones para graficar datos de tablas
import matplotlib.pyplot as plt # funciones generales para graficar
#
# cargamos los datos de IRIS
#
iris_dataset = datasets.load_iris()
#
# como en el cuaderno 1-3, extraemos los datos y etiquetas
#
caracteristicas = iris_dataset['feature_names'] # nombres de las características
muestras = iris_dataset['data'] # datos que describen a cada flor
especies = iris_dataset['target_names'] # nombres de las etiquetas objetivo
etiquetas = iris_dataset['target'] # número que indica la especie de cada flor (lo que queremos predecir)
#
# también importamos el modelo de KNN
from sklearn.neighbors import KNeighborsClassifier # traemos el clasificador de KNN del paquete

## Ejercicio 1: Visualización del dataset

En la primera clase y en el primer cuaderno se señaló la relevancia de visualizar los datos que uno va a analizar. De ello puede extraerse información importante que guíe las estrategias de análisis. Pongamos a prueba la intuición visual con los datos del Iris.

In [ ]:
#Repetimos el código del notebook de la clase 1, donde:

# Separamos el dataset en un conjunto de entrenamiento y otro de testeo
Xtrain, Xtest, ytrain, ytest = model_selection.train_test_split(muestras, etiquetas, random_state=42)
#
# hacemos un scatter plot de las propiedades de las diferentes flores Iris
tabla_iris = pandas.DataFrame(Xtrain, columns=iris_dataset.feature_names)
grr = pandas_plot.scatter_matrix(tabla_iris, c=ytrain, figsize=(10,10))

En el scatter plot anterior, puede ver cómo se distribuyen las diferentes especies de flores según cada par características (indicadas por los nombres en los ejes). Intuitivamente, esperamos que si las especies se ven bien separadas en una gráfica, el rendimiento de un clasificador sea relativamente bueno.

### Parte 1

Según las gráficas realizadas:

*  ¿Qué pares de características espera que generen una mejor clasificación?
*  ¿Cuales espera que generen una peor clasificación? ¿Y una clasificación intermedia?


*Escriba su respuesta aquí*


Ponga a prueba sus hipótesis con el bloque de código abajo, que permite seleccionar las características del Iris que se utilizarán en la clasificación. Explique si sus hipótesis se cumplieron o no.


In [ ]:
# Seleccionamos las variables que usaremos para ajustar el modelo
# Recuerde que las variables posibles son:
# "sepal length (cm)", "sepal width (cm)", "petal length (cm)" y "petal width (cm)"
#
variablesUsadas = ["sepal length (cm)", "petal length (cm)"] # MODIFICABLE

# Extraemos los índices de las columnas que corresponden a esos nombres, usando list comprehensions
indicesUsados = [i for i, val in enumerate(caracteristicas) if val in variablesUsadas]

# Con los índices, armamos Xtrain y Xtest incompletos, con sólo las columnas especificadas
Xtrain_inc = Xtrain[:,indicesUsados]
Xtest_inc = Xtest[:,indicesUsados]

# Ajustamos un modelo KNN a los datos de las columnas seleccionadas
knn = KNeighborsClassifier(n_neighbors=1) # creamos el clasificador
knn.fit(Xtrain_inc, ytrain) # lo ajustamos a los datos

# Evaluamos el rendimiento del modelo en el set de testeo
yPredichos = knn.predict(Xtest_inc) # obtenemos las predicciones del modelo
es_correcto = (yPredichos == ytest) # comparamos con las etiquetas reales, para ver si son correctas
score = 100*np.mean(es_correcto) # porcentaje: 100 x fracción de inferencias correctas
score = np.round(score) # redondeamos

# Imprimimos el resultado
print(f'\n Utilizando las columnas {variablesUsadas}, hay un {score}% de respuestas correctas.\n')

*Escriba su respuesta aquí*

**Extra 1.1:** ¿Puede identificar, a partir de los scatter plots de pares de variables arriba, qué variables mostrarán el mejor y el peor rendimiento cuando son usadas de forma individual?

**Extra 1.2:** Para observar esto de forma mas simple, se requiere un histograma como los de la diagonal, pero donde se vean las tres clases. Graficamos esto usando la funcion `plt.hist()`, vuelva a formular una hipotesis sobre que variable mostrará mejor rendimiento.

Ponga a prueba su estimación.

In [ ]:
variableUsada="sepal length (cm)" # PRUEBEN CON OTRAS!

tabla_iris = pandas.DataFrame(Xtrain, columns=iris_dataset.feature_names)
tabla_iris_target = pandas.DataFrame(ytrain, columns=["label"])

# Grafica 3 histogramas solapados, el parametro alpha le da transparencia a las columnas
plt.hist(tabla_iris[variableUsada][tabla_iris_target["label"]==0], label=[iris_dataset['target_names'][0]], alpha=0.7)
plt.hist(tabla_iris[variableUsada][tabla_iris_target["label"]==1], label=[iris_dataset['target_names'][1]], alpha=0.7)
plt.hist(tabla_iris[variableUsada][tabla_iris_target["label"]==2], label=[iris_dataset['target_names'][2]], alpha=0.7)
plt.legend()
plt.show()

## Ejercicio 2: Análisis de parámetros del modelo

Al aplicar un modelo de aprendizaje automático, es común que se deba elegir algunos parámetros del modelo antes de comenzar.

En el caso de KNN, el modelo tiene el parámetro $K$, que indica cuántos vecinos se utilizan para la clasificación. Hasta ahora venimos utilizando sólo 1 vecino para la clasificación, es decir, clasificamos a cada punto nuevo con la etiqueta de su vecino más cercano.

### Parte 1

* Según su intuición (y puede ayudarse mirando las gráficas), ¿qué espera que ocurra al aumentar el número de vecinos K (desde 1 hasta números grandes)? ¿porqué?

* Ponga a prueba su hipótesis con el bloque de código de abajo.

**Nota:** Dado que el rendimiento con $K=1$ es muy alto para el set de datos Iris, mantenemos lo hecho en el ejercicio 1 de usar sólo dos variables para la clasificación. Así le damos al rendimiento la posibilidad de subir.

In [ ]:
# Seteamos el número de vecinos que usaremos en la clasificación:
nVecinos = 1 # MODIFICABLE

# Preparamos los datos como en el Ejercicio 1, seleccionando dos columnas para dificultarle la tarea al modelo
variablesUsadas = ["sepal width (cm)", "sepal length (cm)"] # MODIFICABLE
indicesUsados   = [i for i, val in enumerate(caracteristicas) if val in variablesUsadas] # buscamos los índices de las columnas con los nombres de arriba
Xtrain_inc = Xtrain[:,indicesUsados] # seleccionamos las columnas con los índices encontrados
Xtest_inc  = Xtest[:,indicesUsados]

# creamos el clasificador, usando la cantidad de vecinos definida
knn = KNeighborsClassifier(n_neighbors = nVecinos)

# Ajustamos el modelo a los datos de las columnas seleccionadas
knn.fit(Xtrain_inc, ytrain)

# Evaluamos el rendimiento del modelo en el set de testeo
yPredichos = knn.predict(Xtest_inc) # obtenemos las predicciones del modelo
es_correcto = (yPredichos == ytest) # comparamos con las etiquetas reales, para ver si son correctas
score = 100*np.mean(es_correcto) # porcentaje: 100 x fracción de inferencias correctas
score = np.round(score) # redondeamos

# chequeamos haber extraído las columnas bien
variablesRealmenteUsadas = [caracteristicas[i] for i in indicesUsados]

# Imprimimos el resultado
print(f'\n Utilizando un número de {nVecinos} vecinos, con las variables {variablesRealmenteUsadas} hay un {score}% de respuestas correctas.\n')

*Escriba su respuesta aquí*

### Parte 2
Para practicar Python, use un loop *for* para *iterar* a través de los valores de 1 a 100 para K, y grafique (o en su defecto imprima) el porcentaje de acierto para cada valor de K.

Sugerencias: \\
1) En el cuaderno 1.1 se explican los loop *for* \\
2) Considere que `range(100)` va desde 0 a 99 (como se menciona en el cuaderno 1.1), y que 0 no es un valor válido para K  \\
3) Puede generar una lista vacía antes del loop, e irla llenando con los valores producidos, usando la función `append` (ver cuaderno 1.1). Ej. si llama a su lista `listita`, puede agregarle el valor de una la variable `valorNuevo` usando `listita.append(valorNuevo)` \\
4) Finalmente, puede graficar el contenido de `listita` usando `matplotlib` como en el cuaderno 1.2

In [ ]:
# Escriba su código aquí:

**Extra 2.1:** ¿Piensa que los resultados del efecto de K pueden depender de las columnas usadas en la clasificación? Si desea, ponga a prueba su hipótesis

**Extra 2.2:** Elegir los parámetros de un modelo (por ejemplo, el K), puede ser una parte importante de un análisis. Conceptualmente, ¿se le ocurre alguna forma de elegir los parámetros "óptimos" para un problema que deba resolver? Piense por ejemplo en el problema de elegir K en el clasificador que estamos usando ahora.



## Ejercicio 3: Pre-procesamiento de datos

Además de la visualización, otra herramienta útil para entender los datos es calcular valores estadisticos como la media y la desviación estándar.

### Parte 1
Calcule la media ($\mu$) y la desviación estándar ($\sigma$) de las 4 características presentes en el Iris dataset (i.e. las 4 columnas de `tabla_iris` o de `muestras`) . Hágalo de las siguientes dos maneras:

1) Con las funciones `mean()` y `std()` que provee el paquete pandas (o `numpy`)

2) aplicando usted las siguientes fórmulas $ \mu = \frac{\sum x_i}{N}$ y $\sigma = \sqrt{\frac{\sum(x_i-\mu)^2}{N}}$. Use para esto último las herramientas del cuaderno 1.2 del teórico (ej. la función `sum()` de `numpy`, o la función `len()`, que le permite obtener el número de elementos N).


In [ ]:
### Escriba su código aquí ###
media =
desviacion =

Muchas veces, el rendimiento de un modelo puede variar según las propiedades estadísticas de los datos utilizados (ej. la media, la desviación estándar, el valor máximo y el valor mínimo de las características). Por ello, es común que antes de aplicar un modelo se haga un pre-procesamiento de los datos, llevándolos a un formato que optimice el rendimiento del modelo.

Un tipo de preprocesamiento muy comun es la estandarización de los datos. La estandarización consiste en que a cada característica se le resta su valor medio $\mu$ y se la divide por la desviación estandar $\sigma$, generando una característica estandarizada con $\mu'=0$ y $\sigma'=1$. Cuando utilizamos varias características diferentes en un modelo, este proceso ayuda a darles a todas la misma *escala*.

### Parte 2
¿Puede explicar intuitivamente porqué puede ser importante que las diferentes variables tengan la misma escala en el modelo de KNN? (piense en un problema donde una característica toma valores entre 0-1, y otra entre 0-1000). Aplique estandarización a todas las columnas, usando la formula $X_{std} = \frac{X-\mu}{\sigma}$ (puede comparar los resultados con los de la funcion del paquete sklearn `preprocessing.scale(X)` que hace lo mismo). Luego, pruebe el modelo de KNN con los datos estandarizados.

**Nota1:** Tengan en cuenta que deben calcular $\mu$ y $\sigma$ para cada columna de los datos de entrenamiento.

**Nota2:** No olviden aplicarle el pre-processamiento a los datos de test tambien! Un detalle no menor es que a los datos de test se le aplica el preprocesamento con los parametros de la base de entrenamiento. Esto es: $X_{std\_test}=\frac{X_{test}-\mu_{train}}{\sigma_{train}}$

In [ ]:
from sklearn import preprocessing
### Escriba su código aquí ###

# Nota: Recuerde que cualquier operacion basica (+,-,/,*) que se le haga a una
# array de datos se aplica a todos los valores del array
ejemplo = np.arange(1,6,step=1)
print(ejemplo)
print(ejemplo - 1)
print(ejemplo / 3)

**Extra 3.1:** Otro método común de preprocesamiento es el de llevar a las variables al rango 0-1, o normalizarlas.

$$X_{norm}=\frac{X-x_{min}}{x_{max}-x_{min}}$$

Pruebe normalizar el dataset Iris y pruebe el rendimiento del modelo KNN.

## Ejercicio Extra 1: Train & Test sets

Al comienzo de este notebook usamos la funcion [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) de sklearn para dividir el Iris dataset, esto divide el dataset de forma tal que, por default, 25% de los datos van para _Xtest_ y el restante 75% se usa para entrenar.

Esto se puede modificar con los parametros *test_size* o *train_size*, junto con otros parametros como *shuffle* (la posibilidad de que el orden de los datos se baraje antes de dividirse en train y test).

¿Como creen que puede afectar esto el rendimiento del modelo KNN? Pueden modificar alguno de estos parametros en la segunda celda del notebook y volver a correr todo para ver que sucede.


## Ejercicio Extra 2: Otro dataset

Para profundizar un poco en las herramietas aprendidas, puede repetir estos ejercicios usando el [Penguins Dataset](https://github.com/allisonhorst/palmerpenguins). Se trata de un dataset con una estructura muy similar a Iris, así que es facil extrapolar lo aprendido para aplicar el modelo KNN a estos datos.

In [ ]:
import seaborn as sns
penguins = sns.load_dataset('penguins')
penguins